In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import json
import time
from dotenv import load_dotenv

In [2]:
user_train_path = '/home/grads/m/mbismay/llm-based-seq-rec/fashion/fashion_data/processed_data/user_train.json'
user_valid_path = '/home/grads/m/mbismay/llm-based-seq-rec/fashion/fashion_data/processed_data/user_valid.json'
user_test_path = '/home/grads/m/mbismay/llm-based-seq-rec/fashion/fashion_data/processed_data/user_test.json'

In [3]:
with open(user_train_path, 'rb') as f:
    user_train_dict = json.load(f)
with open(user_valid_path, 'rb') as f:
    user_valid_dict = json.load(f)
with open(user_test_path, 'rb') as f:
    user_test_dict = json.load(f)

In [4]:
user_id = '1'
print(user_train_dict[user_id])
print(user_valid_dict[user_id])
print(user_test_dict[user_id])

[11, 12, 13, 14, 15]
[12, 13, 14, 15, 16]
[13, 14, 15, 16, 17]


In [5]:
item_information_path = '/home/grads/m/mbismay/llm-based-seq-rec/fashion/fashion_data/processed_data/processed_item_information.json'
with open(item_information_path, 'r') as f:
    item_information_dict = json.load(f)
print(len(item_information_dict))

6089


In [6]:
ratings_path = '../fashion_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1452816000
1,1,0,4.0,1453766400
2,2,0,3.0,1453766400
3,3,0,4.0,1453766400
4,4,0,4.0,1453766400


In [8]:
reasoning_prompt_train_data = "./reasoning_dataset/reasoning_prompt_data/reasoning_prompt_train.pkl"
reasoning_prompt_valid_data = "./reasoning_dataset/reasoning_prompt_data/reasoning_prompt_valid.pkl"

reasoning_train_data = "./reasoning_dataset/reasoning_data/reasoning_train_dict.pkl"
reasoning_valid_data = "./reasoning_dataset/reasoning_data/reasoning_valid_dict.pkl"
reasoning_test_data = "./reasoning_dataset/reasoning_data/reasoning_test_dict.pkl"

In [9]:
with open(reasoning_train_data, 'rb') as f:
    reasoning_train = pickle.load(f)
with open(reasoning_valid_data, 'rb') as f:
    reasoning_valid = pickle.load(f)
with open(reasoning_test_data, 'rb') as f:
    reasoning_test = pickle.load(f)

In [10]:
# print(reasoning_prompt_train[user_id])
print(reasoning_train[user_id])
print(reasoning_valid[user_id])
print(reasoning_test[user_id])


        1. The user seems to prefer lightweight, airy clothing that fits well and looks great. They appreciate good deals and are conscious of the weather, as indicated by their preference for clothing suitable for hot, humid weather. They also seem to prefer stylish, well-designed clothing, as indicated by their purchase of a floral tankini swimsuit and a bodycon dress. However, they are conscious of size and fit, as indicated by their comments about clothing running small.
        2. Based on the user's past behavior, it is likely that they will like the next item, which is a purple top made of chiffon. The user has shown a preference for lightweight, airy clothing, and chiffon is a light, flowing fabric that is well-suited for hot, humid weather. The top is also described as fitting well for people of average height and weight, which aligns with the user's preference for clothing that fits well. While the user has expressed concerns about sizing in the past, the fact that the top i

In [11]:
def get_new_reasoning_text(reasoning):
    # print(f"reasoning: {reasoning}")
    new_reasoning_text = f"Prediction: {reasoning.split(' ')[-1]} \n"
    new_reasoning_text += ' '.join(s.strip() for s in reasoning.split("\n")[:-1])
    # print(f"new_reasoning_text: {new_reasoning_text}")
    return new_reasoning_text

In [12]:
def get_item_description(item_information):
    item_details = ""
    if 'title' in item_information:
        item_details += item_information['title']
    if 'brand' in item_information:
        item_details += f" Brand: {item_information['brand']}"
    if 'price' in item_information:
        item_details += f" Price: {item_information['price']}"
    if 'description' in item_information:
        item_details += f" Description: {item_information['description']}"
    return item_details

In [12]:
### For Train Data
final_train_dict = list()
cnt = 0
max_total_len = 0
avg_total_len = 0
for user, reasoning in reasoning_train.items():
    # print(f"User: {user}")
    cnt += 1
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)

    temp_dict = dict()
    temp_dict["instruction"] = "You are an expert fashion product recommender. Given the list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. "
    prompt = "\n User purchased the following items recently in given order. List of user behaviour for recent items and their description -"
    item_list = user_train_dict[user]
    # print(item_list)
    for item in item_list[:-1]:
        user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    
    target_item = item_list[-1]
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    target_item_rating = user_rating_dict[user_rating_dict['item'] == target_item]['rating'].values[0]
    prompt += f"\nThe user purchased the next target item - {target_item_details}"
    
    prompt += f"""\nDo not use any information not mentioned above. As an expert fashion product recommender, do the following steps -
    Analyze the provided list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    Explain with reasoning whether the user will like or dislike the target item - {item_information_dict[str(target_item)]['title']} in atmost 100 words.
    Finally, Predict whether the user will like the target item or not in the following format - Prediction: Yes or No"""
    temp_dict["input"] = prompt

    temp_dict["output"] = get_new_reasoning_text(reasoning)
    final_train_dict.append(temp_dict)
    # print(final_train_dict)
    total_len = len(temp_dict['instruction'].split()) + len(temp_dict['input'].split()) + len(temp_dict['output'].split())
    avg_total_len += total_len
    max_total_len = max(max_total_len, total_len)
    # break
print("Length of final_train_dict:", len(final_train_dict), final_train_dict[0])
print("Avg Length:", avg_total_len/len(final_train_dict))
print("Max length:", max_total_len)

Length of final_train_dict: 896 {'instruction': 'You are an expert fashion product recommender. Given the list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. ', 'input': '\n User purchased the following items recently in given order. List of user behaviour for recent items and their description -Liked Allegra K Women Boat Neck Batwing Sleeves Pullover Tops Fuchsia M Brand: Allegra K Description: \n\nThe product is a comfortable, stylish, and soft top, available in a great color. However, it has issues with sizing, fitting thin arms, and not being suitable for leggings.\nDisliked Allegra K Ladies Boat Neck Short Dolman Sleeve Cape Style Fashion Top Large / US 14 Khaki Description: \n\nThe product is a cute, warm, and versatile sweater that is comfortable and flattering, but may have issues with sizing and quality of mat

In [13]:
final_train_dict[183]

{'instruction': 'You are an expert fashion product recommender. Given the list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. ',
 'input': '\n User purchased the following items recently in given order. List of user behaviour for recent items and their description -Liked Playboy Watch ACES Playing Cards Bunny Logo Stainless Steel Back Ladies Designer Fashion Authentic Official Licensed Brand: Playboy Description: \nLiked Playboy Watch Pink Swarovski Crystals Bunny Logo Braided Band Stainless Steel Back Ladies Designer Brand: Playboy Description: \nLiked Playboy Ring Solitaire Swarovski Crystal Bunny Charm Platinum Plated Statement Wide Band Size 7 Description: \n\nThe product is a pretty and well-made ring, often complimented by others. However, its platinum color may rub off if it gets wet.\nLiked Playboy Body Jewelry

In [14]:
yes_cnt = 0
no_cnt = 0
for train_data in final_train_dict:
    pred = train_data['output'].split(' ')[1]
    # print(pred)
    if pred == 'Yes':
        yes_cnt += 1
    elif pred == 'No':
        no_cnt += 1
    else:
        print(pred, '--', train_data['output'])
print(yes_cnt, no_cnt, yes_cnt + no_cnt)

671 225 896


In [15]:
final_train_path = "./final_data/fashion/train.json"
with open(final_train_path, 'w') as f:
    json.dump(final_train_dict, f)

In [17]:
### For Valid Data
final_valid_dict = list()
cnt = 0
max_total_len = 0
avg_total_len = 0
for user, reasoning in reasoning_valid.items():
    # print(f"User: {user}")
    cnt += 1
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)

    temp_dict = dict()
    temp_dict["instruction"] = "You are an expert fashion product recommender. Given the list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. "
    prompt = "\n User purchased the following items recently in given order. List of user behaviour for recent items and their description -"
    item_list = user_valid_dict[user]
    # print(item_list)
    for item in item_list[:-1]:
        user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    
    target_item = item_list[-1]
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    target_item_rating = user_rating_dict[user_rating_dict['item'] == target_item]['rating'].values[0]
    prompt += f"\nThe user purchased the next target item - {target_item_details}"
    
    prompt += f"""\nDo not use any information not mentioned above. As an expert fashion product recommender, do the following steps -
    Analyze the provided list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    Explain with reasoning whether the user will like or dislike the target item - {item_information_dict[str(target_item)]['title']} in atmost 100 words.
    Finally, Predict whether the user will like the target item or not in the following format - Prediction: Yes or No"""
    temp_dict["input"] = prompt

    temp_dict["output"] = get_new_reasoning_text(reasoning)
    final_valid_dict.append(temp_dict)
    # print(final_train_dict)
    total_len = len(temp_dict['instruction'].split()) + len(temp_dict['input'].split()) + len(temp_dict['output'].split())
    avg_total_len += total_len
    max_total_len = max(max_total_len, total_len)
    # break
print("Length of final_train_dict:", len(final_valid_dict), final_valid_dict[0])
print("Avg Length:", avg_total_len/len(final_valid_dict))
print("Max length:", max_total_len)

Length of final_train_dict: 896 {'instruction': 'You are an expert fashion product recommender. Given the list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. ', 'input': '\n User purchased the following items recently in given order. List of user behaviour for recent items and their description -Disliked Allegra K Ladies Boat Neck Short Dolman Sleeve Cape Style Fashion Top Large / US 14 Khaki Description: \n\nThe product is a cute, warm, and versatile sweater that is comfortable and flattering, but may have issues with sizing and quality of material.\nLiked Allegra K Woman Boat Neck Button Decor Front Batwing Tunic Top XL Dark Blue Brand: Allegra K Description: \n\nThe product is a comfortable and flattering pullover with a soft, thick material, suitable for fall/winter. However, it may have an awkward sleeve length an

In [18]:
yes_cnt = 0
no_cnt = 0
for valid_data in final_valid_dict:
    pred = valid_data['output'].split(' ')[1]
    # print(pred)
    if pred == 'Yes':
        yes_cnt += 1
    elif pred == 'No':
        no_cnt += 1
    else:
        print(pred, '--', train_data['output'])
print(yes_cnt, no_cnt, yes_cnt + no_cnt)

664 232 896


In [20]:
final_valid_path = "./final_data/fashion/valid.json"
with open(final_valid_path, 'w') as f:
    json.dump(final_valid_dict, f)

In [21]:
user = 121
print(final_train_dict[user])
print(final_valid_dict[user])

{'instruction': 'You are an expert fashion product recommender. Given the list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. ', 'input': '\n User purchased the following items recently in given order. List of user behaviour for recent items and their description -Liked Tagoo Tribal Vintage Round Turquoise Pendant Dangle Earrings(Sea Blue) Description: \n\nProduct Description: This product is a fashionable, high-quality, and stylish choice for bridal parties, as it looks gorgeous and makes the wearer feel confident.\nLiked Guess Factory Women\'s Pepper Leggings Brand: GUESS Description: \n\nProduct Description: These leggings are great for curves, comfortable, soft, and keep legs warm, delivered quickly and beautifully wrapped. No negative features were mentioned in the reviews.\nLiked Waltz-Length Lounger Brand: Ameri

In [13]:
### For Test Data
final_test_dict = list()
cnt = 0
max_total_len = 0
avg_total_len = 0
for user, reasoning in reasoning_test.items():
    # print(f"User: {user}")
    cnt += 1
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)

    temp_dict = dict()
    temp_dict["instruction"] = "You are an expert fashion product recommender. Given the list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. "
    prompt = "\n User purchased the following items recently in given order. List of user behaviour for recent items and their description -"
    item_list = user_test_dict[user]
    # print(item_list)
    for item in item_list[:-1]:
        user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    
    target_item = item_list[-1]
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    target_item_rating = user_rating_dict[user_rating_dict['item'] == target_item]['rating'].values[0]
    prompt += f"\nThe user purchased the next target item - {target_item_details}"
    
    prompt += f"""\nDo not use any information not mentioned above. As an expert fashion product recommender, do the following steps -
    Analyze the provided list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    Explain with reasoning whether the user will like or dislike the target item - {item_information_dict[str(target_item)]['title']} in atmost 100 words.
    Finally, Predict whether the user will like the target item or not in the following format - Prediction: Yes or No"""
    temp_dict["input"] = prompt

    temp_dict["output"] = get_new_reasoning_text(reasoning)
    final_test_dict.append(temp_dict)
    # print(final_train_dict)
    total_len = len(temp_dict['instruction'].split()) + len(temp_dict['input'].split()) + len(temp_dict['output'].split())
    avg_total_len += total_len
    max_total_len = max(max_total_len, total_len)
    # break
print("Length of final_train_dict:", len(final_test_dict), final_test_dict[0])
print("Avg Length:", avg_total_len/len(final_test_dict))
print("Max length:", max_total_len)

Length of final_train_dict: 896 {'instruction': 'You are an expert fashion product recommender. Given the list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. ', 'input': '\n User purchased the following items recently in given order. List of user behaviour for recent items and their description -Liked Allegra K Woman Boat Neck Button Decor Front Batwing Tunic Top XL Dark Blue Brand: Allegra K Description: \n\nThe product is a comfortable and flattering pullover with a soft, thick material, suitable for fall/winter. However, it may have an awkward sleeve length and uneven seams, with inconsistent sizing and color.\nLiked Allegra K Women Sleeveless Sheer Mesh Yoke Mini Skater Dress L Black Brand: Allegra K Description: \n\nThe product is a cute, form-fitting dress made of good material, with stretch and give. However, it

In [14]:
yes_cnt = 0
no_cnt = 0
for test_data in final_test_dict:
    pred = test_data['output'].split(' ')[1]
    # print(pred)
    if pred == 'Yes':
        yes_cnt += 1
    elif pred == 'No':
        no_cnt += 1
    else:
        print(pred, '--', test_data['output'])
print(yes_cnt, no_cnt, yes_cnt + no_cnt)

669 227 896


In [18]:
user = 121
# print(final_train_dict[user])
# print(final_valid_dict[user])
print(final_test_dict[user])

{'instruction': 'You are an expert fashion product recommender. Given the list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. ', 'input': '\n User purchased the following items recently in given order. List of user behaviour for recent items and their description -Liked Waltz-Length Lounger Brand: AmeriMark Description: \n\nThe product is a comfortable and roomy lounger, made of soft polyester material. It fits well, is easy to wash, and is available in large sizes. However, some customers may find the material unattractive and the color options limited.\nLiked Mudder 5 Sizes 925 Silver Plated Snake Chain Necklace, 10 Pieces Description: \n\nThe product is a set of two 16" necklaces, made of thick, continuous and coiled metal, which are great for the money but may turn color and have rust issues.\nDisliked Muchmore Wom

In [19]:
final_test_path = "./final_data/fashion/test.json"
with open(final_test_path, 'w') as f:
    json.dump(final_test_dict, f)